# Otimização do Modelo WDO
Otimização dos parâmetros e validação cruzada do modelo.

In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path(os.path.dirname(os.getcwd()))
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

plt.style.use('seaborn')
pd.set_option('display.max_columns', None)

## 1. Grid Search de Parâmetros

In [ ]:
# Carrega dados
from src.data.loaders.market_data_loader import MarketDataLoader
from src.ml.features.feature_engineering import FeatureEngineering
from src.agents.market_agent import RiskManagementAgent, MarketRegimeAgent
from src.agents.agent_coordinator import AgentCoordinator
from src.backtest.monte_carlo import MonteCarloSimulator

# Configuração dos parâmetros para teste
param_grid = {
    'risk_management': {
        'max_drawdown': [-0.03, -0.04, -0.05, -0.06],
        'max_position_size': [2, 3, 4, 5]
    },
    'market_regime': {
        'window_size': [10, 15, 20, 25],
        'volatility_threshold': [0.1, 0.15, 0.2]
    }
}

# Carrega e prepara dados
db_path = PROJECT_ROOT / 'src' / 'data' / 'database' / 'candles.db'
loader = MarketDataLoader(str(db_path))
data = loader.load_data()

engineer = FeatureEngineering()
features = engineer.create_technical_features(data)
features = engineer.create_temporal_features(features)

In [ ]:
# Função para avaliar uma combinação de parâmetros
def evaluate_parameters(features, risk_params, regime_params):
    risk_agent = RiskManagementAgent(**risk_params)
    regime_agent = MarketRegimeAgent(**regime_params)
    coordinator = AgentCoordinator([risk_agent, regime_agent])
    
    # Validação cruzada com 5 folds
    tscv = TimeSeriesSplit(n_splits=5)
    metrics_list = []
    
    for train_idx, test_idx in tscv.split(features):
        train_data = features.iloc[train_idx]
        test_data = features.iloc[test_idx]
        
        # Simula com dados de teste
        simulator = MonteCarloSimulator(n_simulations=100)
        returns = test_data['returns'].dropna()
        simulation = simulator.simulate_returns(
            historical_returns=returns,
            initial_capital=100000
        )
        
        # Calcula métricas
        metrics = {
            'sharpe_ratio': simulation.sharpe_ratio,
            'max_drawdown': simulation.max_drawdown,
            'profit_factor': simulation.profit_factor,
            'win_rate': simulation.win_rate
        }
        metrics_list.append(metrics)
    
    # Média das métricas entre os folds
    avg_metrics = pd.DataFrame(metrics_list).mean()
    return avg_metrics

# Grid search
results = []
for max_dd in param_grid['risk_management']['max_drawdown']:
    for pos_size in param_grid['risk_management']['max_position_size']:
        for window in param_grid['market_regime']['window_size']:
            for vol_thresh in param_grid['market_regime']['volatility_threshold']:
                risk_params = {
                    'max_drawdown': max_dd,
                    'max_position_size': pos_size
                }
                regime_params = {
                    'window_size': window,
                    'volatility_threshold': vol_thresh
                }
                
                metrics = evaluate_parameters(features, risk_params, regime_params)
                results.append({
                    **risk_params,
                    **regime_params,
                    **metrics.to_dict()
                })

results_df = pd.DataFrame(results)
print('
Melhores parâmetros por métrica:')
print('
Melhor Sharpe Ratio:')
print(results_df.loc[results_df['sharpe_ratio'].idxmax()])

print('
Menor Max Drawdown:')
print(results_df.loc[results_df['max_drawdown'].idxmin()])

## 2. Validação dos Melhores Parâmetros

In [ ]:
# Usando os melhores parâmetros encontrados
best_params_idx = results_df['sharpe_ratio'].idxmax()
best_params = results_df.loc[best_params_idx]

risk_agent = RiskManagementAgent(
    max_drawdown=best_params['max_drawdown'],
    max_position_size=best_params['max_position_size']
)

regime_agent = MarketRegimeAgent(
    window_size=best_params['window_size'],
    volatility_threshold=best_params['volatility_threshold']
)

coordinator = AgentCoordinator([risk_agent, regime_agent])

# Teste final com dados mais recentes
test_size = int(len(features) * 0.2)  # 20% mais recentes
train_data = features[:-test_size]
test_data = features[-test_size:]

simulator = MonteCarloSimulator(n_simulations=1000)
returns = test_data['returns'].dropna()
simulation = simulator.simulate_returns(
    historical_returns=returns,
    initial_capital=100000
)

print('
Resultados Finais com Parâmetros Otimizados:')
print(f'Sharpe Ratio: {simulation.sharpe_ratio:.2f}')
print(f'Max Drawdown: {simulation.max_drawdown:.2%}')
print(f'Win Rate: {simulation.win_rate:.2%}')
print(f'Profit Factor: {simulation.profit_factor:.2f}')